### Alignment
The first script is the approach taken from D. Portugal-Barron. Unfortunately, I see to have missed a step here, or my approach, in which I substituted HISAT for Bowtie2, simply does not work. The second script is based on the following approach: https://www.metagenomics.wiki/tools/samtools/breadth-of-coverage.

In [ ]:
#define parameters:
SAMPLENAME=
PATTERNR1=name_pattern_R1
PATTERNR2=name_pattern_R2
GENOMEPATH=/path/to/genome/file/
BOWTIE2_INDEXES=/path/to/indices
READSPATH=/path/to/reads
TEMPDIR=/temp/storage/dir/$SAMPLENAME/
OUTDIR=/dir/for/results/$SAMPLENAME/
#check whether indicated paths are correct

[ -d $READSPATH ] || { echo 'Invalid path to mapping files, exiting' && exit; }
[ -d $INDEXPATH ] || { echo 'Invalid path to contigs, exiting' && exit; }
[ -d $GENOMEPATH ] || { echo 'Invalid path to contigs, exiting' && exit; }

mkdir $OUTDIR
mkdir $TEMPDIR
for f in <name1> <name2> <name3>
do

#align reads to reference
bowtie2 -x $BOWTIE2_INDEXES/"$f"_Index -1 $READSPATH/$PATTERNR1 -2 $READSPATH/$PATTERNR2 -S $TEMPDIR/"$f"_to_"$SAMPLENAME".sam
#convert SAM to BAM
samtools view -bS $TEMPDIR/"$f"_to_"$SAMPLENAME".sam > $TEMPDIR/"$f"_to_"$SAMPLENAME".bam
#sort BAM file
samtools sort $TEMPDIR/"$f"_to_"$SAMPLENAME".bam -o $OUTDIR/"$f"_to_"$SAMPLENAME".sorted.bam 
done; 

#then use samtools depth to estimate depth
samtools depth $OUTDIR/*_to_"$SAMPLENAME".sorted.bam -o $OUTDIR/"$SAMPLENAME"_depth.txt
#then use samtools coverage to estimate coverage
samtools coverage *_to_"$SAMPLENAME".sorted.bam -o $OUTDIR/"SAMPLENAME"_coverage.txt


In [ ]:
#define parameters:
SAMPLENAME=
BOWTIE2_INDEXES=/path/to/indices
READSPATH=/path/to/reads
TEMPDIR=/temp/storage/dir/$SAMPLENAME/
OUTDIR=/dir/for/results/$SAMPLENAME/

[ -d $READSPATH ] || { echo 'Invalid path to mapping files, exiting' && exit; }
[ -d $INDEXPATH ] || { echo 'Invalid path to contigs, exiting' && exit; }
[ -d $GENOMEPATH ] || { echo 'Invalid path to contigs, exiting' && exit; }

mkdir $OUTDIR
mkdir $TEMPDIR

cat $READSPATH/"$SAMPLENAME"_*.fq > "$SAMPLENAME"_reads.fq

for f in Brevminu Durus Mcav SymClado SymKawa SymMicro SymNata SymNec SymPilo
do

echo 'These are the values for the depth of coverage at various target depths. First row is depth at 1X, then 5, 10, 15, 20, and 25. Last line is the target genome length.' > $OUTDIR/"$f"_"$SAMPLENAME".txt
bowtie2 -x $INDEXPATH/"$f""_Index --no-unal -U $TEMPDIR/"$SAMPLENAME"_reads.fq -S "$f"_"$SAMPLENAME".sam -p $NSLOTS
samtools view -bS $TEMPDIR/"$f"_"$SAMPLENAME".sam > $TEMPDIR/"$f"_"$SAMPLENAME".bam
samtools sort $TEMPDIR/"$f"_"$SAMPLENAME".bam > $TEMPDIR/"$f"_"$SAMPLENAME".sorted.bam
samtools index $TEMPDIR/"$f"_"$SAMPLENAME".sorted.bam 
samtools mpileup "$f"_"$SAMPLENAME".sorted.bam | awk -v X=1 '$0>=X' | wc -l >> $OUTDIR/"$f"_"$SAMPLENAME".txt
samtools mpileup "$f"_"$SAMPLENAME".sorted.bam | awk -v X=5 '$4>=X' | wc -l >> $OUTDIR/"$f"_"$SAMPLENAME".txt
samtools mpileup "$f"_"$SAMPLENAME".sorted.bam | awk -v X=10 '$9>=X' | wc -l >> $OUTDIR/"$f"_"$SAMPLENAME".txt
samtools mpileup "$f"_"$SAMPLENAME".sorted.bam | awk -v X=15 '$14>=X' | wc -l >> $OUTDIR/"$f"_"$SAMPLENAME".txt
samtools mpileup "$f"_"$SAMPLENAME".sorted.bam | awk -v X=20 '$19>=X' | wc -l >> $OUTDIR/"$f"_"$SAMPLENAME".txt
samtools mpileup "$f"_"$SAMPLENAME".sorted.bam | awk -v X=25 '$24>=X' | wc -l >> $OUTDIR/"$f"_"$SAMPLENAME".txt
bowtie2-inspect -s "$f"_Index | awk '{ FS = "\t" } ; BEGIN{L=0}; {L=L+$3}; END{print L}' >> $OUTDIR/"$f"_"$SAMPLENAME".txt
echo 'Depth of coverage at each given depth can be calculated by dividing the number of bases by the number of bases in the target genome.' >> $OUTDIR/"$f"_"$SAMPLENAME".txt
done